In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
import os

from sklearn.metrics import classification_report
import tensorflow.keras.backend as K
from tensorflow.keras import layers, regularizers

E0000 00:00:1754907546.087340      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


In [ ]:

MAX_SEQ_LEN = 150
EMBEDDING_DIM = 768
LABEL_COLS = ['Prolongation', 'Block', 'SoundRep', 'WordRep', 'Interjection']
NUM_CLASSES = len(LABEL_COLS)

BATCH_SIZE_PER_REPLICA = 32
EPOCHS = 400
LEARNING_RATE = 1e-4
THRESHOLD = 0.5

In [3]:
print("--- Initializing TPU Strategy ---")

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu='local')
    # tf.config.experimental_connect_to_cluster(tpu)
    # tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    
    print('TPU successfully initialized. Found TPU: ', tpu.master())
    print("Number of replicas:", strategy.num_replicas_in_sync)

except Exception as e:
    print(f"TPU initialization failed, this is the error: {e}")
    print("Falling back to default strategy (CPU/GPU).")
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA

--- Initializing TPU Strategy ---
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1754907581.468595      10 service.cc:148] XLA service 0x5aa30a9a7070 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754907581.468643      10 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1754907581.468647      10 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1754907581.468650      10 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1754907581.468653      10 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1754907581.468656      10 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1754907581.468659      10 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1754907581.468661      10 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1754907581.468664      10 service.cc:156]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
print(f"Global batch size set to: {BATCH_SIZE}")

Global batch size set to: 256


In [ ]:
with strategy.scope():
    print("\nBuilding StutterFold Model")

    def conformer_block(x, num_heads=4, ff_dim=128, dropout=0.1):
        # Conv Module
        x_conv = layers.LayerNormalization()(x)
        x_conv = layers.Conv1D(filters=EMBEDDING_DIM * 2, kernel_size=1, activation="relu")(x_conv)
        x_conv = layers.DepthwiseConv1D(kernel_size=3, padding="same")(x_conv)
        x_conv = layers.BatchNormalization()(x_conv)
        x_conv = layers.Activation("swish")(x_conv)
        x_conv = layers.Conv1D(filters=EMBEDDING_DIM, kernel_size=1, kernel_regularizer=regularizers.l1(5e-5))(x_conv) # L1
        x_conv = layers.Dropout(dropout)(x_conv)
        x = x + x_conv

        # Attn Module
        x_attn = layers.LayerNormalization()(x)
        x_attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=EMBEDDING_DIM // num_heads)(x_attn, x_attn)
        x_attn = layers.Dropout(dropout)(x_attn)
        x = x + x_attn

        # FF Module
        x_ff = layers.LayerNormalization()(x)
        x_ff = layers.Dense(ff_dim, activation="swish")(x_ff)
        x_ff = layers.Dropout(dropout)(x_ff)
        x_ff = layers.Dense(EMBEDDING_DIM)(x_ff)
        x = x + x_ff

        return x
    def build_stutterfold_model(input_shape, num_classes):
        inputs = layers.Input(shape=input_shape)
        x = inputs

        # Module 1: Context Refinement using Conformer blocks
        x = conformer_block(x, num_heads=4, ff_dim=1028, dropout=0.4)
        x = conformer_block(x, num_heads=4, ff_dim=1028, dropout=0.4)
        # At this point, x is the enriched feature sequence: (Batch, 150, 768)

        # Module 2: Stuttering Event Prediction
        # Applyies a classifier to every single time step independently
        x = layers.TimeDistributed(
            layers.Dense(num_classes, activation='sigmoid', kernel_regularizer=regularizers.l1(5e-5))    # L1
        )(x)

        outputs = layers.GlobalAveragePooling1D()(x)
        
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        return model
    model = build_stutterfold_model(input_shape=(MAX_SEQ_LEN, EMBEDDING_DIM), num_classes=NUM_CLASSES)

    optimizer = tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=1e-4)
    
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall'), tf.keras.metrics.AUC(name='auc_roc'), tf.keras.metrics.AUC(curve='PR', name='auc_pr')]
    )

model.summary()

print(f"LEARNING RATE: {LEARNING_RATE}")
print(f"BATCH SIZE: {BATCH_SIZE}")


Building StutterFold Model


I0000 00:00:1754907726.248311      10 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 150, 768)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 150, 768)  │      1,536 │ input_layer[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 150, 1536) │  1,181,184 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv1d    │ (None, 150, 1536) │      6,144 │ conv1d[0][0]      │
│ (DepthwiseConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 150, 1536) │      6,144 │ depthwise_conv1d… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 150, 1536) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 150, 768)  │  1,180,416 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 150, 768)  │          0 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 150, 768)  │          0 │ input_layer[0][0… │
│                     │                   │            │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 150, 768)  │      1,536 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 150, 768)  │  2,362,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 150, 768)  │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 150, 768)  │          0 │ add[0][0],        │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 150, 768)  │      1,536 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 150, 1028) │    790,532 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 150, 1028) │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 150, 768)  │    790,272 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 150, 768)  │          0 │ add_1[0][0],      │
│                     │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 150, 768)  │      1,536 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 12,647,181 (48.25 MB)

 Trainable params: 12,641,037 (48.22 MB)

 Non-trainable params: 6,144 (24.00 KB)

LEARNING RATE: 0.0001
BATCH SIZE: 256


In [11]:
print("\nLoading all compressed FluencyBank features")

Fluency_data = np.load('/kaggle/input/fluencyprocc/fluencybank_test_data.npz')

# Access the arrays by the keys we used when saving ('x' and 'y')
X_test, y_test = Fluency_data['x'], Fluency_data['y']


print("All compressed data loaded successfully.")
print(f"X_train shape: {X_test.shape}")
print(f"y_train shape: {y_test.shape}")


Loading all compressed FluencyBank features
All compressed data loaded successfully.
X_train shape: (1751, 150, 768)
y_train shape: (1751, 6)


In [12]:
# Remove the last column ('NoStutter') from the labels
y_test = y_test[:, :-1]

# Filter out samples that now have NO labels
test_indices = np.where(y_test.sum(axis=1) > 0)[0]

X_test, y_test = X_test[test_indices], y_test[test_indices]

print("Data loaded and modified successfully.")
print(f"New X_test shape: {X_test.shape}")

Data loaded and modified successfully.
New X_test shape: (1357, 150, 768)


In [13]:

print(f"New X_test shape: {y_test.shape}")

New X_test shape: (1357, 5)


In [15]:
print("\n Verifying final training set distribution after augmentation")

fluency_test_labels_df = pd.DataFrame(y_test, columns=LABEL_COLS)

print("Final distribution of labels in the training set (including augmented samples):")
print(fluency_test_labels_df.sum().sort_values(ascending=False))


 Verifying final training set distribution after augmentation
Final distribution of labels in the training set (including augmented samples):
Interjection    734
Block           664
SoundRep        471
WordRep         418
Prolongation    388
dtype: int64


In [16]:
print("\nBuilding tf.data pipeline")

def prepare_dataset(X, y, shuffle=False, drop_remainder = False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    ds = ds.cache()
    if shuffle:
        ds = ds.shuffle(buffer_size=2048)
    ds = ds.batch(BATCH_SIZE, drop_remainder=drop_remainder)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

test_dataset = prepare_dataset(X_test, y_test, shuffle=False, drop_remainder=True)
print("Datasets created successfully.")


Building tf.data pipeline
Datasets created successfully.


In [ ]:
model.load_weights('/kaggle/working/out_stFold/best_run4_drpt.keras')
print("\n--- Evaluating Model on the Test Set ---")
y_pred_probs = model.predict(test_dataset)
y_pred_binary = (y_pred_probs > THRESHOLD).astype(int)

num_predictions = len(y_pred_binary)
y_test_eval = y_test[:num_predictions]

print("\n--- Classification Report (5 Stuttering Classes) ---")
report = classification_report(y_test_eval, y_pred_binary, target_names=LABEL_COLS, zero_division=0)
print(report)


--- Evaluating Model on the Test Set ---


I0000 00:00:1754908192.214435      10 encapsulate_tpu_computations_pass.cc:266] Subgraph fingerprint:7682831442896535194
E0000 00:00:1754908192.369672      10 meta_optimizer.cc:966] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node functional_1/batch_normalization_1/Cast/ReadVariableOp.
I0000 00:00:1754908192.636312    1031 tpu_compilation_cache_interface.cc:442] TPU host compilation cache miss: cache_key(4515627853124701595), session_name()
I0000 00:00:1754908197.488887    1031 tpu_compile_op_common.cc:245] Compilation of 4515627853124701595 with session name  took 4.852530721s and succeeded
I0000 00:00:1754908197.502601    1031 tpu_compilation_cache_interface.cc:476] TPU host compilation cache: compilation complete for cache_key(4515627853124701595), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_7682831442896535194", property.function_library_fingerprint = 17562501376178834605, property.mlir_module_f

5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 184ms/step

--- Classification Report (5 Stuttering Classes) ---
              precision    recall  f1-score   support

Prolongation       0.50      0.50      0.50       375
       Block       0.55      0.59      0.57       610
    SoundRep       0.55      0.66      0.60       432
     WordRep       0.48      0.47      0.48       389
Interjection       0.84      0.69      0.75       715

   micro avg       0.60      0.60      0.60      2521
   macro avg       0.58      0.58      0.58      2521
weighted avg       0.61      0.60      0.60      2521
 samples avg       0.60      0.62      0.57      2521

